In [1]:
# import Earth Engine packages
import ee
import geemap
import numpy as np

In [2]:
# initialize EE
try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize! Try to Authenticate EE')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

The Earth Engine package initialized successfully!


In [3]:
print ("Please select an Area of Interest using the Drawing tools")
Map = geemap.Map(center=[50, -115], zoom=6)
Map

Please select an Area of Interest using the Drawing tools


Map(center=[50, -115], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [4]:
#Define the Area of interest
AOI = ee.FeatureCollection(Map.draw_features)
AOI_geometry = ee.Geometry.Polygon (AOI.geometry().getInfo()['coordinates'])
Centroid_Coord = AOI_geometry.centroid().getInfo()['coordinates']
print (Centroid_Coord)
print (AOI_geometry.getInfo()['coordinates'])

[-117.29114900000047, 49.49916177329421]
[[[-117.37191, 49.4517], [-117.210388, 49.4517], [-117.210388, 49.546598], [-117.37191, 49.546598], [-117.37191, 49.4517]]]


In [5]:
#Define dates
date_start = '2020-04-15'
date_end= '2020-05-14'

In [6]:
#Define a Cloud Threshold
cloud_threshold = 30

In [7]:
#Setup a function to caclulate the NDSI
def CalculateNDSI(image):
    NDSI = image.normalizedDifference(['B3', 'B11'])\
                .rename('NDSI')
    return image.addBands(NDSI)
print ("NDSI band created!")

NDSI band created!


In [8]:
#Setup a function to caclulate the Cloud and Cloud Shadow Mask
def CloudMask (image):
    cloud_mask = image.expression(
      "((b('MSK_CLDPRB') >= 90)) || ((b('SCL') == 3)) ? 2 " +
       ": ((b('MSK_CLDPRB') >= 50) && (b('B8A') >= 3000)) || ((b('MSK_CLDPRB') >= 20) && (b('B8A') >= 9000))  ? 1" +
         ": 0").rename('CloudMask')
    return image.addBands(cloud_mask)
print ("Cloud Mask band created!")

Cloud Mask band created!


In [9]:
#Add Sentinel-2 Collection and filter using AOI, dates, cloud threshold. Select the latest image.
S2 = ee.ImageCollection("COPERNICUS/S2_SR")\
      .filterDate(date_start, date_end)\
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_threshold))\
      .filterBounds(AOI)\
      .map(CloudMask)\
      .map(CalculateNDSI)

#Check how many images are returned by the Query
count_images = S2.size().getInfo()
print("The Sentinel-2 query returned", count_images, "images")

The Sentinel-2 query returned 2 images


In [10]:
#Mosaic the retunred images if the area of interest is covering multiple acquisitons
if count_images == 1:
    image = S2.first()
    image_date = image.date().format('YYYY-MM-dd').getInfo()
    image_bands = image.bandNames().getInfo()
    print ("The image was acquired on", image_date, "and has the following bands:",image_bands)
    
elif count_images > 1:
    count_images = S2.size().getInfo()
    image = S2.mosaic()
    print ("A mosaic of most recent images is created!")

else:
    print ("No images returned! Modify the acquisition dates or lower the cloud threshold!")

A mosaic of most recent images is created!


In [11]:
#Create the Snow Cover Extent (SCE) layer
SCE = image.expression(
      "((b('CloudMask') == 0 && (b('NDSI') >= 0.3) && (b('B4') >= 1000))) ? 2" +
       ": (b('CloudMask') > 0) ? 1" +
        ": 0")\
    .clip(AOI)

SCE_masked = SCE.updateMask(SCE.gt(0))
print ("SCE layer created!")

SCE layer created!


In [15]:
#Create a Band Composite image (SWIR2,SWIR1,Green)
BandComposite = image.select('B4', 'B3', 'B2')\
                  .clip(AOI)
print ("Band Composite image (SWIR2,SWIR1,Green) created!")

Band Composite image (SWIR2,SWIR1,Green) created!


In [16]:
#Set the visualisation parameters.
SCEViz = {
  "min": 0,
  "max": 2,
  "palette": ['yellow', 'red','blue'],
}

BandCompViz = {
  "min": 0,
  "max": 1500,
  "gamma": [0.95, 1.1, 1]
}

In [17]:
#Setup a split Screen Map and visualize results.
right_layer = geemap.ee_tile_layer(SCE_masked, SCEViz, 'Snow Cover Extent')
left_layer = geemap.ee_tile_layer(BandComposite, BandCompViz, 'Sentinel-2 SWIR composite')
Map2 = geemap.Map(center= Centroid_Coord[::-1], zoom=10) 
Map2.split_map(left_layer, right_layer)

#Add a legend.
legend_keys = ['Snow', 'Clouds']
legend_colors = ['#0000FF', '#FF0000']

Map2.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')

Map2

Map(center=[49.49916177329421, -117.29114900000047], controls=(ZoomControl(options=['position', 'zoom_in_text'…